In [1]:
import os


In [2]:
%pwd


'c:\\Users\\Achraf\\Desktop\\MASTER\\Python Avancé\\Mini-Projet\\mlops-fakenews-text-classification\\research'

In [3]:
os.chdir("../")


In [4]:
%pwd


'c:\\Users\\Achraf\\Desktop\\MASTER\\Python Avancé\\Mini-Projet\\mlops-fakenews-text-classification'

In [5]:
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/achrafHR/mlops-fakenews-text-classification.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="achrafHR"
os.environ["MLFLOW_TRACKING_PASSWORD"]="1186be9494033fd5bc54deb4eb5754a45febe492"

In [6]:
from sklearn.linear_model import LogisticRegression
from fakeNewsClassifier.utils.common import load_bin
from pathlib import Path

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Achraf\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Achraf\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [7]:
model = load_bin(Path("artifacts/training/trained_model.pkl"))


[2025-05-14 00:22:36,715: INFO: common: binary file loaded from: artifacts\training\trained_model.pkl]


In [8]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    processed_x_test: Path
    processed_y_test: Path
    mlflow_uri: str
    params: dict
    

In [9]:
from fakeNewsClassifier.constants import *
from fakeNewsClassifier.utils.common import read_yaml, create_directories, save_json, load_bin


In [14]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    
    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model=Path("artifacts/training/trained_model.pkl"),
            processed_x_test=Path("artifacts/data_preprocessing/X_test_tfidf.pkl"),
            processed_y_test=Path("artifacts/data_preprocessing/y_test.pkl"),
            mlflow_uri="https://dagshub.com/achrafHR/mlops-fakenews-text-classification.mlflow",
            params=self.params.get("model params", {}),
        )
        return eval_config

In [15]:
from pathlib import Path
import mlflow
import mlflow.sklearn
from urllib.parse import urlparse
from sklearn.metrics import accuracy_score, log_loss

In [ ]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    
    def get_test_data(self):
        self.X_test = load_bin(self.config.processed_x_test)
        self.y_test = load_bin(self.config.processed_y_test)
    
    def evaluation(self):
        self.model = load_bin(self.config.path_of_model)
        # Pour régression logistique
        self.y_pred = self.model.predict(self.X_test)
        self.y_pred_proba = self.model.predict_proba(self.X_test)
        
        # Calcul des métriques pour régression logistique
        
        self.accuracy = accuracy_score(self.y_test, self.y_pred)
        self.loss = log_loss(self.y_test, self.y_pred_proba)
        self.score = [self.loss, self.accuracy]  
        
        self.save_score()

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

    
    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        
        with mlflow.start_run():
            mlflow.log_params(self.config.params)
            mlflow.log_metrics(
                {"loss": self.score[0], "accuracy": self.score[1]}
            )
            # Model registry does not work with file store
            if tracking_url_type_store != "file":

                # Register the model
                # There are other ways to use the Model Registry, which depends on the use case,
                # please refer to the doc for more information:
                # https://mlflow.org/docs/latest/model-registry.html#api-workflow
                mlflow.sklearn.log_model(self.model, "model", registered_model_name="LogisticRegression")
            else:
                mlflow.sklearn.log_model(self.model, "model")

In [19]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.get_test_data()
    evaluation.evaluation()
    evaluation.log_into_mlflow()

except Exception as e:
   raise e

[2025-05-14 00:28:52,583: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-05-14 00:28:52,588: INFO: common: yaml file: params.yaml loaded successfully]
[2025-05-14 00:28:52,590: INFO: common: created directory at: artifacts]
[2025-05-14 00:28:52,609: INFO: common: binary file loaded from: artifacts\data_preprocessing\X_test_tfidf.pkl]
[2025-05-14 00:28:52,618: INFO: common: binary file loaded from: artifacts\data_preprocessing\y_test.pkl]
[2025-05-14 00:28:52,621: INFO: common: binary file loaded from: artifacts\training\trained_model.pkl]
[2025-05-14 00:28:52,644: INFO: common: json file saved at: scores.json]


2025/05/14 00:29:15 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Successfully registered model 'LogisticRegression'.
2025/05/14 00:29:20 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: LogisticRegression, version 1
Created version '1' of model 'LogisticRegression'.


🏃 View run resilient-bug-690 at: https://dagshub.com/achrafHR/mlops-fakenews-text-classification.mlflow/#/experiments/0/runs/cbcc159d8bcc40d7bbc91f58b7719c4d
🧪 View experiment at: https://dagshub.com/achrafHR/mlops-fakenews-text-classification.mlflow/#/experiments/0
